### 什么是文本挖掘？
- 数据不光是0-9这样的数字，数据是信息的载体
- 非数字类的数据难以做到定量化；总信息量过大，人工难以承受；次要信息难以压缩，有效信息难以提取；难以纳入数学分析框架
- 文本挖掘意义：
- NLP流程：
    - 1. 原始文本：网页文本、新闻、报告
    - 2. 分词：英文->空格，中文->无
    - 3. 清洗：无用标签、特殊符号、停用词、大写转小写
    - 4. 标准化：go,going,went->go
    - 5. 特征提取：string->向量
    - 6. 建模：相似度算法、分类算法等（机器学习算法）
    

#### 分词
- 分词工具：Jieba分词、SnowNLP、LTP、HanNLP等

- 分词

In [2]:
import jieba

text = "征战四海只为今日一胜，我不会再败了。"
# cut(sentence,cut_all)：返回：generator类型
seg = jieba.cut(text)
print("/".join(seg))

# lcut()：返回list类型
seg = jieba.lcut(text)
print(seg)

征战/四海/只/为/今日/一胜/，/我/不会/再败/了/。
['征战', '四海', '只', '为', '今日', '一胜', '，', '我', '不会', '再败', '了', '。']


- 词性分析
    - posseg.cut(text)：返回generator
    - posseg.lcut(text)：返回list

In [21]:
import jieba.posseg as posseg

text = "征战四海只为今日一胜，我不会再败了。"
seg = posseg.cut(text)
print([se for se in seg])

# 直接返回列表形式
seg = posseg.lcut(text)
print(seg)


[pair('征战', 'v'), pair('四海', 'ns'), pair('只', 'd'), pair('为', 'p'), pair('今日', 't'), pair('一', 'm'), pair('胜', 'v'), pair('，', 'x'), pair('我', 'r'), pair('不会', 'v'), pair('再败', 'v'), pair('了', 'ul'), pair('。', 'x')]
[pair('征战', 'v'), pair('四海', 'ns'), pair('只', 'd'), pair('为', 'p'), pair('今日', 't'), pair('一', 'm'), pair('胜', 'v'), pair('，', 'x'), pair('我', 'r'), pair('不会', 'v'), pair('再败', 'v'), pair('了', 'ul'), pair('。', 'x')]


- 关键词抽取
    - 基于TF-IDF算法
    - 基于TextRank算法

In [38]:
import jieba.analyse as analyse

text = "征战四海只为今日一胜，我不会再败了。"
# TF-IDF算法
rf_result = analyse.extract_tags(text,topK=3)   # topK：指关键词数量，默认20
print(rf_result)

# TextRank
tr_result = analyse.textrank(text,topK=3)   # topK：指关键词数量，默认20
print(tr_result)

['一胜', '再败', '征战']
['征战', '再败', '四海']


- 分词：
    - 精确模式：最常用的分词方法
    - 全模式：所有可能的词都列举出来
    - 搜索引擎模式：适用于搜索引擎使用

In [75]:
text = "今天天气真好"
# 精确模式
accurateModel = jieba.cut(text,cut_all=False)
# 全模式
fullModel = jieba.cut(text,cut_all=True)
# 搜索引擎模式
searchModel = jieba.cut_for_search(text)
print(' '.join(accurateModel))
print(' '.join(fullModel))
print(' '.join(searchModel))


# 直接返回list
accurateModel = jieba.lcut(text,cut_all=False)
# 全模式
fullModel = jieba.lcut(text,cut_all=True)
# 搜索引擎模式
searchModel = jieba.lcut_for_search(text)
print(accurateModel)
print(fullModel)
print(searchModel)

今天天气 真 好
今天 今天天气 天天 天气 真好
今天 天天 天气 今天天气 真 好
['今天天气', '真', '好']
['今天', '今天天气', '天天', '天气', '真好']
['今天', '天天', '天气', '今天天气', '真', '好']


- 增加词语
    - 自定义字典：load_userdict(file_name)，一个词占一行；每一行分三部分：词语、词频（可省略）、词性（可省略），用空格隔开
    - 增加词语：add_word(word)
    - 删除词语：del_word(word) 
    - 调节单个词语的词频：suggest_freq(segment,tune=True)：调节单个词语的词频，使其不能被分开

In [73]:
jieba.add_word("李编辑") # 在程序中动态修改词典
jieba.suggest_freq("不喜欢",tune=True)
jieba.suggest_freq("不高兴",tune=True)

text = "李编辑如果不去北京，那么他就会不喜欢也不高兴"
seg = jieba.lcut(text)
print(seg)

['李编辑', '如果', '不去', '北京', '，', '那么', '他', '就', '会', '不喜欢', '也', '不高兴']


#### 拼写纠错
- 用户输入->候选->编辑距离
    - 之前方法：用户输入->从词典中训导编辑距离最小的->返回
    - 现在方法：用户输入->生成编辑距离为1，2的字符串->过滤->返回
- 如何过滤？

- 停用词过滤、出现频率低的词汇过滤
    - Stemming、Lemmatization

#### 文本表示
词典：[我们，去，爬山，今天，你们，昨天，跑步]
 
每个单词的表示：<font color='red'>one-hot方法，不统计频率，只要出现则为1</font>
 
我们：(1,0,0,0,0,0,0) -> 7维=|词典|

爬山：(0,0,1,0,0,0,0) -> 7维=|词典|

跑步：(0,0,0,0,0,0,1) -> 7维=|词典|

昨天：(0,0,0,0,0,1,0) -> 7维=|词典|

词典：[我们，又，去，爬山，今天，你们，昨天，跑步]

每个句子的表示：<font color='red'>Boolean方法，不统计频率，只要出现则为1</font>

我们 今天 去 爬山：(1,0,1,1,1,0,0,0) -> 8维=|词典|

你们 昨天 跑步：(0,0,0,0,0,1,1,1) -> 8维=|词典|

你们 又 去 爬山 又 去 跑步：(0,1,1,1,0,1,0,1) -> 8维=|词典|

词典：[我们，又，去，爬山，今天，你们，昨天，跑步]

每个句子的表示：<font color='red'>count-based方法，统计频率</font>

我们 今天 去 爬山：(1,0,1,1,1,0,0,0) -> 8维=|词典|
,
你们 昨天 跑步：(0,0,0,0,0,1,1,1) -> 8维=|词典|

你们 又 去 爬山 又 去 跑步：(0,2,2,1,0,1,0,1) -> 8维=|词典|




#### 文本相似度
<font color='red'>计算距离（欧式距离）：d = |s1 - s2|。距离越大，个体差异越大</font>

s1：我们 今天 去 爬山：(1,0,1,1,1,0,0,0)

s2：你们 昨天 跑步：(0,0,0,0,0,1,1,1)

s3：你们 又 去 爬山 又 去 跑步：(0,2,2,1,0,1,0,1)

d(s1,s2) = 根号(1²+1²+1²+1²+1²+1²+1²) = 根号(7)

d(s1,s3) = 根号(1²+2²+1²+1²+1²+1²) = 根号(9)

d(s2,s3) = 根号(2²+2²+1²+1²) = 根号(10)

结论：sim(s1,s2)>sim(s1,s3)>sim(s2,s3)

<font color='red'>计算相似度（余弦相似度）：d = (s1·s2)/(|s1|·|s2|)</font>

s1：我们 今天 去 爬山：(1,0,1,1,1,0,0,0)

s2：你们 昨天 跑步：(0,0,0,0,0,1,1,1)

s3：你们 又 去 爬山 又 去 跑步：(0,2,2,1,0,1,0,1)

d(s1,s2) = (0)/() = 0

d(s1,s3) = (3)/(2*根号(11)) =3/2根号(11)

d(s2,s3) = (2)/(根号(3)*根号(11)) =2/根号(33)

结论：sim(s1,s3)>sim(s2,s3)>sim(s1,s2)

#### tf-idf 文本表示
- <font color='red'> tfidf(w) = tf(d,w) * idf(w) </font>
    - tf(d,w)：文档d中w的词频
    - idf(w)：考虑单词的重要性，log(N/N(w))
    - N：语料库中的文档总数
    - N(w)：词语w出现在多少个文档

词典：[今天，上，NLP，课程，的，有，意思，数据，也]

|词典| = 9

句子1：今天 上 NLP 课程

(1·log(3/2),1·log(3/1),1·log(3/1),1·log(3/3),0,0,0,0,0) = (log(3/2),log3,log3,0,0,0,0,0,0) <font color='red'> -> tfidf向量 </font>

句子2：今天 的 课程 有 意思

(1·log(3/2),0,0,1·log(3/3),1·log(3/1),1·log(3/2),1·log(3/2),0,0) = (log(3/2),0,0,0,log3,log(3/2),log(3/2),0,0)

句子3：数据 课程 也 有 意思

(0,0,0,1·log(3/3),0,1·log(3/2),1·log(3/2),1·log(3/1),1·log(3/1)) = (0,0,0,0,0,log(3/2),log(3/2),log3,log3)

#### 词向量
<font color='red'> 利用Ont-hot表示法表达单词之间相似度？NO </font>

<font color='red'> Distributed Representation：分布式表示方法 </font>

分布式表示法的长度是人工定义

我们：[0.1,0.2,0.4,0.2]

爬山：[0.2,0.3,0.7,0.1]

运动：[0.2,0.3,0.6,0.2]

昨天：[0.5,0.9,0.1,0.3]

欧氏距离：

d（我们，爬山）= 根号（0.1²+0.1²+0.3²+0.1²）= 根号（0.12）

d（运动，爬山）= 根号（0.02）

d（运动，爬山）< d（我们，爬山） => sim（运动，爬山）> sim（我们，爬山）

#### 学习词向量

<font color='red'> 从词向量到句子向量 </font>

例如：

我们 = （0.1，0.2，0.1，0.3）

去 = （0.3，0.2，0.15，0.2）

运动 = （0.2，0.15，0.4，0.7）

“我们去运动” 怎么获得句子向量？

方法：Average法则（最简单的方法）

对3个词向量进行叠加，再进行平均，得到“我们去运动”的句子向量 = （0.2，0.18，0.22，0.4）

#### 倒排表（Inverted Index）
- 在搜索引擎中有每个文件都对应一个文件ID，文件内容被表示为一系列关键词的集合（实际上在搜索引擎索引库中，关键词也已经转换为关键词ID）。例如“文档1”经过分词，提取了20个关键词，每个关键词都会记录它在文档中的出现次数和出现位置。得到正向索引的结构如下：

    - 1、“文档1”的ID > 单词1：出现次数，出现位置列表；单词2：出现次数，出现位置列表；…………。

    - 2、“文档2”的ID > 此文档出现的关键词列表。

Doc1:我们 今天 运动

Doc2:我们 昨天 运动

Doc3:我们 上 课

Doc4:我们 上 什么 课

词典：[我们，今天，运动，昨天，上，课，什么]

我们：[Doc1,Doc2]

今天：[Doc1]

运动：[Doc1,Doc2]

昨天：[Doc2]

上：[Doc3,Doc4]

课：[Doc3,Doc4]

什么：[Doc4]

假设通过百度查询：运动，那么返回Doc1和Doc2

#### Noisy Channel Model

p(text|source) = p(source|text)·p(text)/p(source) （贝叶斯公式）p(source)是一个常数，那么 p(text|source) ∝ p(source|text)·p(text)

- 机器翻译
    - eg：英->中，p(中|英) ∝ p(英|中)·p(中)，p(英|中)是翻译模型，p(中)是语言模型

- 语言识别
    - p(文本|语言信息) ∝ p(语言信息|文本)·p(文本)，p(语言信息|文本)是翻译识别模型，p(文本)是语言模型

- 密码破解
    - p(明文|暗文) ∝ p(暗文|明文)·p(明文)，p(明文)是语言模型

#### 语言模型
- 作用：用来判断是否一句话在语法上通顺

比较：今天是周日 VS 今天周日是

- 目标：Compute the probability of a sentence or sequence of words.

#### Chain Rule

条件概率：p(AB) = p(A|B)·p(B) = p(B|A)·p(A)

P(ABCD) = P(A)·P(A|B)·P(C|AB)·P(D|ABC)

P(今天,是,春节,我们,都,休息) = P(今天)·P(是|今天)·P(春节|今天,是)·P(我们|今天,是,春节)·P(都|今天,是,春节,我们)·P(休息|今天,是,春节,我们,休息)

#### Markov假设

1st order Markov assumption：P(休息|今天,是,春节,我们,都) ≈ P(休息|都)

2st order Markov assumption：P(休息|今天,是,春节,我们,都) ≈ P(休息|我们,都)

3st order Markov assumption：P(休息|今天,是,春节,我们,都) ≈ P(休息|春节,我们,都)

...

#### 语言模型

- Unigram

P(ABCD) = P(A)·P(B)·P(C)·P(D)，变量之间相互独立

P(今天,是,春节,我们,都,休息) = P(今天)·P(是)·P(春节)·P(我们)·P(都)·P(休息)

P(今天,春节,是,都,我们,休息) = P(今天)·P(春节)·P(是)·P(都)·P(我们)·P(休息)

- Bigram，基于1st order Markov assumption

P(今天,是,春节,我们,都,休息) = P(今天)·P(是|今天)·P(春节|是)·P(我们|春节)·P(都|我们)·P(休息|都)

P(今天,春节,是,都,我们,休息) = P(今天)·P(春节|今天)·P(是|春节)·P(都|是)·P(我们|都)·P(休息|我们)

- N-gram

N≥3

#### 估计语言模型的概率

- Unigram：Estimating Probability

语料库：

今天 的 天气 很好 啊

我 很 想 出去 运动

但 今天 上午 有 课程

训练营 明天 才 开始

V = 18，去除重复的单词

P(今天 开始 训练营 课程) = P(今天)·P(开始)·P(训练营)·P(课程) = 2/18·1/18·1/18·1/18 = 2/104976

P(今天 没有 训练营 课程) = P(今天)·P(没有)·P(训练营)·P(课程) = 2/18·0/18·1/18·1/18 = 0

- Bigram：Estimating Probability，Bigram基于1st order Markov assumption

语料库：

今天 的 天气 很好 啊

我 很 想 出去 运动

但 今天 上午 有 课程

训练营 明天 才 开始

V = 18，去除重复的单词

P(今天 上午 想 出去 运动) = P(今天)·P(上午|今天)·P(想|上午)·P(出去|想)·P(运动|出去) = 2/18·1/2·1·1/2·1 = 1/38

P(今天 上午 的 天气 很好 呢) = P(今天)·P(上午|今天)·P(的|上午)·P(天气|的)·P(很好|天气)·P(呢|很好) = 0

- N-gram：Estimating Probability

N = 3 时，

语料库：

今天 上午 的 天气 很好 啊

我 很 想 出去 运动

但 今天 上午 有 课程

训练营 明天 才 开始

V = 18，去除重复的单词

P(今天 上午 有 课程) = P(今天)·P(上午|今天)·P(有|今天,上午)·P(课程|上午,有) = 2/18·1/2·1/2·1 = 1/40

P(今天 没有 训练营 课程) = P(今天)·P(没有|今天)·P(训练营|今天,没有)·P(课程|没有,训练营) = 2/18·0·... = 0

#### 评估语言模型

Q：训练出来的语言模型效果好还是坏？

  理想情况下：
  
  1、假设有两个语言模型A，B
  
  2、选定一个特定的任务比如乒协纠错

  3、把两个模型A，B都应用在此任务中
  
  4、最后比较准确率，从而判断A，B的表现

<font color='red'>Perplexity = 2⁻ˣ，x:average log likelihood（平均对数似然）</font>

<font color='red'>Perplexity越小越好</font>

训练好的Bigram：

P(天气|今天) = 0.01

P(今天) = 0.002

P(很好|天气) = 0.1

P(适合|很好) = 0.01

P(出去|适合) = 0.02

P(运动|出去) = 0.1


今天：P(今天) = 0.002（计算出来为likelihodd） -> log P(今天) = a1（转换成 log likelihood）

今天天气：P(天气|今天) = 0.01（计算出来为likelihodd） -> log P(气|今天) = a2（转换成 log likelihood）

今天天气很好：P(很好|天气) = 0.1（计算出来为likelihodd） -> log P(很好|天气) = a3（转换成 log likelihood）

今天天气很好，适合：P(适合|很好) = 0.01（计算出来为likelihodd） -> log P(适合|很好) = a4（转换成 log likelihood）

今天天气很好，适合出去：P(出去|适合) = 0.02（计算出来为likelihodd） -> log P(出去|适合) = a5（转换成 log likelihood）

今天天气很好，适合出去运动：P(运动|出去) = 0.1（计算出来为likelihodd） -> log P(运动|出去) = a6（转换成 log likelihood）

计算average log likelihood，即 x = (a1+a2+a3+a4+a5+a6)/6

最后计算Perplexity = 2⁻ˣ

#### Smoothing：平滑方法：处理概率为0

语料库：Bigram

今天 上午 的 天气 很好 啊

我 很 想 出去 运动

但 今天 上午 有 课程

训练营 明天 才 开始

V = 18,去除重复的单词

P(今天 训练营 没有) = P(今天)·P(训练营|今天)·P(没有|训练营) = 2/18·0·... = 0

P(今天 没有 训练营 课程) = 2/18·0·... = 0

如何解决上述概率为0情况？答：Add-one Smoothing、Add-K Smoothing、Interpolation、Good-Turing Smoothing

<font color='red'>Add-one Smoothing</font>

Pᴍʟᴇ(wᵢ|wᵢ₋₁) = c(wᵢ₋₁,wᵢ)/c(wᵢ₋₁)，存在缺陷，若分母为0，则P=0

改进：Pᴍʟᴇ(wᵢ|wᵢ₋₁) = ( c(wᵢ₋₁,wᵢ) + 1 )/( c(wᵢ₋₁) + V )，V是词典大小（去除重复单词）

<font color='red'>Add-K Smoothing</font>

Pᴍʟᴇ(wᵢ|wᵢ₋₁) = ( c(wᵢ₋₁,wᵢ) + k )/( c(wᵢ₋₁) + kV )

K怎么选择？1、k=1,2,3,...,100；2、优化

<font color='red'>Interpolation</font>

核心思路：在计算Trigram概率时，同时考虑Unigram,Bigram,Trigram出现的频次。

P(wₙ|wₙ₋₁,wₙ₋₂) = λ₁·P(wₙ|wₙ₋₁,wₙ₋₂)+λ₂·P(wₙ|wₙ₋₁)+λ₃·P(wₙ)，λ₁+λ₂+λ₃ = 1
